In [ ]:
from langchain.document_loaders import WebBaseLoader

from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import pandas as pd
import joblib
import re
import copy
import yaml
import os

from pprint import pprint

In [ ]:
os.environ["OPENAI_API_KEY"] = yaml.safe_load(open('../credentials.yml'))['openai']

OPENAI_LLM = 'gpt-3.5-turbo'
# OPENAI_LLM = 'gpt-4o'

In [ ]:
url = "https://university.business-science.io/p/4-course-bundle-machine-learning-and-web-applications-r-track-101-102-201-202a"

# Create a document loader for the website
loader = WebBaseLoader(url)

# Load the data from the website
documents = loader.load()

In [ ]:
pprint(documents[0].page_content)

('\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '4-Course Bundle - Machine Learning and Web Applications (R-Track) | Bu\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Toggle navigation\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Business Science\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Login\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Sign Up\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\xa0\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '4-Course Bundle - Machine Learning + Expert Web Applications (R-Track)\n'
 '\n'
 '\n'
 'Go from Beginner to Expert Data Scientist & Shiny Developer in Under '
 '6-Months\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'off original price!\

In [ ]:
df = pd.read_csv("data/data-rag-product-information/products.csv")

df['website']

loader = WebBaseLoader(df['website'].tolist())

documents = loader.load()

documents[1].metadata

{'source': 'https://university.business-science.io/p/5-course-bundle-machine-learning-web-apps-time-series',
 'title': '5 Course Bundle - Machine Learning + Web Apps + Time Series | Business',
 'description': 'The Full 5-Course R-Track System',
 'language': 'No language found.'}

In [ ]:
joblib.load("data/data-rag-product-information/products.pkl")

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n4-Course Bundle - Machine Learning and Web Applications (R-Track) | Bu\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n\n\n\nBusiness Science\n\n\n\n\nLogin\n\n\n\n\nSign Up\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\n4-Course Bundle - Machine Learning + Expert Web Applications (R-Track)\n\n\nGo from Beginner to Expert Data Scientist & Shiny Developer in Under 6-Months\n\n\n\n\n\n\n\n\n\n\n\n\n\noff original price!\n\n\nThe coupon code you entered is expired or invalid, but the course is still available!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGet Four of our Premium R Courses that Build Expert-Level Machine Learning Skills & Expert Web Application Skills\n\n\nLearn the foundations of data science and how to apply machine learning to return business value\n\n\n\nTaking these three courses is the equivalent of:\n\n200+ tool-based courses\n\n4 end-

In [ ]:
# * Clean the Beautiful Soup Page Content

def clean_text(text):

    text = re.sub(r'\n+', '\n', text) 
    text = re.sub(r'\s+', ' ', text)  

    text = re.sub(r'Toggle navigation.*?Business Science', '', text, flags=re.DOTALL)
    text = re.sub(r'© Business Science University.*', '', text, flags=re.DOTALL)

    # Replace encoded characters
    text = text.replace('\xa0', ' ')
    text = text.replace('ðŸŽ‰', '')  

    # Extract relevant content
    relevant_content = []
    lines = text.split('\n')
    for line in lines:
        if any(keyword in line for keyword in ["Enroll in Course", "data scientist", "promotion", "salary", "testimonial"]):
            relevant_content.append(line.strip())

    # Join the relevant content back into a single string
    cleaned_text = '\n'.join(relevant_content)

    return cleaned_text

In [ ]:
# Test cleaning a single document

pprint(documents[1].page_content)

('\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '5 Course Bundle - Machine Learning + Web Apps + Time Series | Business\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Toggle navigation\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Business Science\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Login\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'Sign Up\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\xa0\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '5 Course R-Track: Machine Learning, Web Apps, & Time Series\n'
 '\n'
 '\n'
 'The Full 5-Course R-Track System\n'
 '\n'
 '\n'
 '\xa0\xa0Enroll in Course\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '\n'
 'off original price!\n'
 '\n'
 '\n'
 'T

In [ ]:
pprint(clean_text(documents[1].page_content))

('5 Course Bundle - Machine Learning + Web Apps + Time Series | Business  '
 'Login Sign Up 5 Course R-Track: Machine Learning, Web Apps, & Time Series '
 'The Full 5-Course R-Track System Enroll in Course off original price! The '
 'coupon code you entered is expired or invalid, but the course is still '
 'available! Achieve your goals, accelerate your career, become the data '
 "scientist you were meant to be. Here's what Jeremy thought of Matt's R-Track "
 'Program "Matt is an absolutely incredible teacher" "His depth of knowledge '
 'and teaching style was what I needed" "The culture Matt fosters helps '
 'students learn from each other" "Matt has given me quite a bit of career '
 'advice as well" "His courses are worth every penny" Everything You Need to '
 'Become the Expert Business Data Scientist for Your Organization 1,848 '
 'Lessons, 152.9 Hours of Video, and 24 Challenges to Test Your Skills Enroll '
 'in Course The proof is in the results These are the results that our '
 

In [ ]:
documents_clean = copy.deepcopy(documents)

for document in documents_clean:
    document.page_content = clean_text(document.page_content)
    
documents_clean

[Document(page_content="4-Course Bundle - Machine Learning and Web Applications (R-Track) | Bu  Login Sign Up 4-Course Bundle - Machine Learning + Expert Web Applications (R-Track) Go from Beginner to Expert Data Scientist & Shiny Developer in Under 6-Months off original price! The coupon code you entered is expired or invalid, but the course is still available! Get Four of our Premium R Courses that Build Expert-Level Machine Learning Skills & Expert Web Application Skills Learn the foundations of data science and how to apply machine learning to return business value Taking these three courses is the equivalent of: 200+ tool-based courses 4 end-to-end projects 4 frameworks 1 foundational data science education with tidyverse 1 advanced machine learning with H2O + business consulting education 1 foundational web application education with shiny 1 advanced web application + production with shiny + AWS Courses Included with Purchase Data Science for Business Part 1 Your Data Science Jou

In [ ]:
# Assess Length

for document in documents_clean:
    print(document.metadata)
    print(len(document.page_content))
    print("---")

{'source': 'https://university.business-science.io/p/4-course-bundle-machine-learning-and-web-applications-r-track-101-102-201-202a', 'title': '4-Course Bundle - Machine Learning and Web Applications (R-Track) | Bu', 'description': 'Go from Beginner to Expert Data Scientist & Shiny Developer in Under 6-Months', 'language': 'No language found.'}
5771
---
{'source': 'https://university.business-science.io/p/5-course-bundle-machine-learning-web-apps-time-series', 'title': '5 Course Bundle - Machine Learning + Web Apps + Time Series | Business', 'description': 'The Full 5-Course R-Track System', 'language': 'No language found.'}
13022
---
{'source': 'https://university.business-science.io/p/data-science-starter-bundle-r-track-courses-101-102', 'title': 'Data Science Starter Bundle: R-Track - Courses 101 & 102 | Business', 'description': 'Learn the skills needed to become build and distribute intermediate web applications in R', 'language': 'No language found.'}
4354
---
{'source': 'https:/

In [ ]:
# * Chunk into 500-1000 using Recursive Splitter

CHUNK_SIZE = 1000

text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size = CHUNK_SIZE,
    chunk_overlap=100,
)

documents_clean_recursive = text_splitter_recursive.split_documents(documents_clean)

documents_clean_recursive

len(documents_clean_recursive)

273

In [ ]:
documents_clean_recursive

[Document(page_content='4-Course Bundle - Machine Learning and Web Applications (R-Track) | Bu  Login Sign Up 4-Course Bundle - Machine Learning + Expert Web Applications (R-Track) Go from Beginner to Expert Data Scientist & Shiny Developer in Under 6-Months off original price! The coupon code you entered is expired or invalid, but the course is still available! Get Four of our Premium R Courses that Build Expert-Level Machine Learning Skills & Expert Web Application Skills Learn the foundations of data science and how to apply machine learning to return business value Taking these three courses is the equivalent of: 200+ tool-based courses 4 end-to-end projects 4 frameworks 1 foundational data science education with tidyverse 1 advanced machine learning with H2O + business consulting education 1 foundational web application education with shiny 1 advanced web application + production with shiny + AWS Courses Included with Purchase Data Science for Business Part 1 Your Data Science Jou

In [ ]:
embedding_function = OpenAIEmbeddings(
    model='text-embedding-ada-002',
)

In [ ]:
vectorstore = Chroma.from_documents(
    documents_clean_recursive, 
    embedding=embedding_function, 
    persist_directory="data/data-rag-product-information/products_recursive.db"
)

vectorstore

In [ ]:
vectorstore_1 = Chroma(
    embedding_function=embedding_function, 
    persist_directory="data/data-rag-product-information/products_recursive.db"
)

vectorstore_1

In [ ]:
vectorstore_1.similarity_search("Is the 4-Course R-Track Open for Enrollment?", k = 4)

[Document(page_content='5 Course Bundle - Machine Learning + Web Apps + Time Series | Business  Login Sign Up 5 Course R-Track: Machine Learning, Web Apps, & Time Series The Full 5-Course R-Track System Enroll in Course off original price! The coupon code you entered is expired or invalid, but the course is still available! Achieve your goals, accelerate your career, become the data scientist you were meant to be. Here\'s what Jeremy thought of Matt\'s R-Track Program "Matt is an absolutely incredible teacher" "His depth of knowledge and teaching style was what I needed" "The culture Matt fosters helps students learn from each other" "Matt has given me quite a bit of career advice as well" "His courses are worth every penny" Everything You Need to Become the Expert Business Data Scientist for Your Organization 1,848 Lessons, 152.9 Hours of Video, and 24 Challenges to Test Your Skills Enroll in Course The proof is in the results These are the results that our 5-Course R-Track Students a

In [ ]:
retriever_1 = vectorstore_1.as_retriever()

retriever_1

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8d2ae2fc70>)

In [ ]:
# Connect to the Vector Store (Run all other times)
vectorstore_2 = Chroma(
    embedding_function=embedding_function, 
    persist_directory="data/data-rag-product-information/products_clean.db"
)

vectorstore_2

retriever_2 = vectorstore_2.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# * LLM Specification

model = ChatOpenAI(
    model = OPENAI_LLM,
    temperature = 0.7,
)

In [ ]:
rag_chain_1 = (
    {"context": retriever_1, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
result = rag_chain_1.invoke("Is the 4-Course R-Track Open for Enrollment?")

ValidationError: 1 validation error for Document
page_content
  none is not an allowed value (type=type_error.none.not_allowed)

In [ ]:
rag_chain_2 = (
    {"context": retriever_2, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
result = rag_chain_2.invoke("Is the 4-Course R-Track Open for Enrollment?")

In [ ]:
vectorstore_1 = Chroma(
    embedding_function=embedding_function, 
    persist_directory="data/data-rag-product-information/products_recursive.db"
)

In [ ]:
retriever_1 = vectorstore_1.as_retriever()

In [ ]:
rag_chain_1 = (
    {"context": retriever_1, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
result = rag_chain_1.invoke("Is the 4-Course R-Track Open for Enrollment?")

ValidationError: 1 validation error for Document
page_content
  none is not an allowed value (type=type_error.none.not_allowed)

In [ ]:
vectorstore_1.similarity_search("Is the 4-Course R-Track Open for Enrollment?", k = 4)

[Document(page_content='5 Course Bundle - Machine Learning + Web Apps + Time Series | Business  Login Sign Up 5 Course R-Track: Machine Learning, Web Apps, & Time Series The Full 5-Course R-Track System Enroll in Course off original price! The coupon code you entered is expired or invalid, but the course is still available! Achieve your goals, accelerate your career, become the data scientist you were meant to be. Here\'s what Jeremy thought of Matt\'s R-Track Program "Matt is an absolutely incredible teacher" "His depth of knowledge and teaching style was what I needed" "The culture Matt fosters helps students learn from each other" "Matt has given me quite a bit of career advice as well" "His courses are worth every penny" Everything You Need to Become the Expert Business Data Scientist for Your Organization 1,848 Lessons, 152.9 Hours of Video, and 24 Challenges to Test Your Skills Enroll in Course The proof is in the results These are the results that our 5-Course R-Track Students a

In [ ]:
retriever_1

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f8d09e36f50>)

In [ ]:
result = rag_chain_2.invoke("Is the 4-Course R-Track Open for Enrollment?")

In [ ]:
pprint(result)

'No, the 4-Course R-Track is closed for enrollment.'


In [ ]:
result = rag_chain_2.invoke("What courses are included in the 5-Course R Track?")

pprint(result)

('The courses included in the 5-Course R Track are:\n'
 '1. Data Science for Business Part 1\n'
 '2. Data Science for Business Part 2\n'
 '3. Shiny Web Applications Part 1\n'
 '4. Shiny Web Applications Part 2\n'
 '5. High Performance Time Series')


In [ ]:
result = rag_chain_2.invoke("What courses are included in the 4-Course R Track?")

pprint(result)

('The courses included in the 4-Course R Track are:\n'
 '1. Data Science for Business Part 1\n'
 '2. Data Science for Business Part 2\n'
 '3. Shiny Web Applications Part 1\n'
 '4. Shiny Web Applications Part 2')


In [ ]:
result = rag_chain_2.invoke("What is Learning Labs PRO?")

pprint(result)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 23340 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
OPENAI_LLM = 'gpt-4o'

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# * LLM Specification

model = ChatOpenAI(
    model = OPENAI_LLM,
    temperature = 0.7,
)

In [ ]:
rag_chain_2 = (
    {"context": retriever_2, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
result = rag_chain_2.invoke("What is Learning Labs PRO?")

pprint(result)

('Learning Labs PRO is a subscription-based service offered by Business '
 'Science University that provides access to a library of over 80 one-hour '
 'advanced data science projects, including live coding and demonstrations. '
 'Subscribers receive a new one-hour course each month on intermediate and '
 'advanced topics related to data science, covering areas that are not '
 'typically included in the core R and Python track courses. The service aims '
 'to provide continuous education on critical data science topics and includes '
 'access to code, video instruction, and Shiny applications. Members can '
 'cancel the subscription at any time from the account management page.')
